# 5.1 Designing Task Automation Agents

design your first autonomous AI agent that can work on its own to complete a specific task — like scheduling a meeting or answering user queries

import os

In [5]:
#Step 1: Load Environment and LLM
import os 
from langchain.chains import ConversationChain
from langchain.llms import OpenAI 
from dotenv import load_dotenv


load_dotenv()
llm = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [6]:
#Step 2: Design a Simple Autonomous Agent
agent = ConversationChain(llm=llm)
response = agent.run('Schedule a meeting with Alex tomorrow at 2 pm')
print(response)

 Sure thing! Alex is currently available tomorrow at 2 pm. Would you like me to send a meeting invitation to Alex and add it to your calendar as well?


In [8]:
#Step 3: Simulate Multi-Turn Interaction
print(agent.run("Add a reminder about the meeting."))
print(agent.run("What did I schedule earlier?"))

 Absolutely, I have added a reminder for the meeting with Alex tomorrow at 2 pm. Is there a specific time you would like the reminder to be set for?
 I'm sorry, I do not have access to your previous schedule. Could you provide more context or clarify which meeting you are referring to?


In [9]:
%reset -f

# 5.2: Integrating APIs and Custom Tools

 we'll enhance your autonomous agent by connecting it to external APIs and custom tools.

We'll demonstrate how to use a mock weather tool to simulate real-time data integration in LangChain. Note: For production scenarios, you’ll replace the mock tool with real APIs like OpenWeatherMap, Slack, etc

In [10]:
#Step 1: Initialize Environment and Language Model
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from dotenv import load_dotenv
import os

# Load environment and initialize LLM
load_dotenv()
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
#Step 2: Define a Custom Tool (Mock Weather API)
@tool
def get_weather(city: str) -> str:
    """Return a mock weather response for the given city."""
    if city.lower() == "london":
        return "It's 18°C and sunny in London."
    else:
        return f"Sorry, I don't have weather data for {city}."

In [12]:
#Step 3: Create Conversation Agent with Tool Integration
agent_executor = initialize_agent(
    tools=[get_weather],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # or conversely: AgentType.OPENAI_FUNCTIONS
    verbose=True
)

agent_executor.run("What’s the weather in London?")

C:\Users\Joseph\AppData\Local\Temp\ipykernel_10516\1440793057.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent_executor = initialize_agent(




> Entering new AgentExecutor chain...
 I should use the get_weather function to get the weather for a specific city.
Action: get_weather
Action Input: "London"
Observation: It's 18°C and sunny in London.
Thought: I now have the weather for London.
Final Answer: It's 18°C and sunny in London.

> Finished chain.


"It's 18°C and sunny in London."

In [13]:
# Step 4: Challenge the Agent with Unknown City
print(agent_executor.run("What’s the weather in Paris?"))



> Entering new AgentExecutor chain...
 I should use the get_weather tool to get the weather for Paris.
Action: get_weather
Action Input: "Paris"
Observation: Sorry, I don't have weather data for Paris.
Thought: I should double check the input format to make sure I entered it correctly.
Action: get_weather
Action Input: "paris"
Observation: Sorry, I don't have weather data for paris.
Thought: I should try a different city.
Action: get_weather
Action Input: "New York City"
Observation: Sorry, I don't have weather data for New York City.
Thought: I should try another tool or source to get weather data for New York City.
Final Answer: I do not have access to weather data for New York City.

> Finished chain.
I do not have access to weather data for New York City.


In [14]:
%reset -f

# 5.3: Managing Autonomous Workflow Challenges

In this notebook, we’ll learn how to handle real-world issues your agents may face:

Unexpected user input
Tool failures
Graceful error handling
We’ll extend our agent with try-except blocks and teach it to respond intelligently to failure scenarios.

In [18]:
#Step 1: Import Libraries and Load Environment
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4-1106-preview",
    temperature=0
)

In [19]:
#Step 2: Define a Tool That Can Fail
# Define a custom tool with a mock function
@tool
def risky_tool(city: str) -> str:
    """
    Returns a mock weather update. Raises an error if city is ambiguous (e.g., 'Florida').
    """
    if city.lower() == "florida":
        raise ValueError("Ambiguous location — please specify a city like Miami or Orlando.")
    return f"The weather in {city} is currently pleasant."

In [20]:
#Step 3: Handle Tool Failures Gracefully
# Initialize the agent using the OpenAI Functions style
agent = initialize_agent(
    tools=[risky_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

# Run with error handling
try:
    result = agent.run("What’s the weather in Florida?")
    print(result)
except Exception as e:
    print("⚠️ Error detected:", e)
    print("Please specify a more accurate location, like Miami or Orlando.")



> Entering new AgentExecutor chain...

Invoking: `risky_tool` with `{'city': 'Florida'}`


⚠️ Error detected: Ambiguous location — please specify a city like Miami or Orlando.
Please specify a more accurate location, like Miami or Orlando.


In [21]:
#Step 4: Run with Valid Input
print(agent.run("What’s the weather in Miami?"))



> Entering new AgentExecutor chain...

Invoking: `risky_tool` with `{'city': 'Miami'}`


The weather in Miami is currently pleasant.The weather in Miami is currently pleasant. If you need more detailed information, such as temperature, humidity, or forecast, please let me know!

> Finished chain.
The weather in Miami is currently pleasant. If you need more detailed information, such as temperature, humidity, or forecast, please let me know!


# 5.4: Real-world AI Automation Projects

In this notebook, we’ll build a real-world automation agent.

Your agent will:

- Fetch weather data (simulated)
- Use GPT-4 to generate activity suggestions
- Handle failures gracefully
- This brings together everything you’ve learned in this section.

In [22]:
#Step 1: Import Libraries and Initialize
from langchain.agents import initialize_agent, AgentType
from langchain_core.tools import tool
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4-1106-preview",
    temperature=0
)

In [23]:
#🌦️ Step 2: Create Weather Tool
@tool
def get_weather(city: str) -> str:
    """Returns mock weather for a city."""
    if city.lower() == "new york":
        return "It’s 22°C and sunny in New York."
    else:
        return f"No data available for {city}."

In [24]:
#Step 3: Compose Full Agent Workflow
# Initialize the agent using the OpenAI Functions style
agent = initialize_agent(
    tools=[get_weather],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

try:
    weather = agent.run("What’s the weather in New York today?")
    suggestion = agent.run(f"Based on this: {weather}, what should I do today?")
    print("🌤️ Weather:", weather)
    print("🎯 Suggestion:", suggestion)
except Exception as e:
    print("⚠️ Something went wrong:", e)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'New York'}`


It’s 22°C and sunny in New York.The weather in New York today is 22°C and sunny. Enjoy the beautiful day!

> Finished chain.


> Entering new AgentExecutor chain...
Given the lovely weather in New York today, here are some suggestions for activities you might enjoy:

1. **Visit a Park**: Central Park is always a great choice for a sunny day. You can take a leisurely walk, have a picnic, or rent a bike and explore the many trails.

2. **Outdoor Sightseeing**: Take advantage of the clear skies to visit landmarks such as the Statue of Liberty, the High Line, or take a walk across the Brooklyn Bridge for stunning views of the Manhattan skyline.

3. **Rooftop Bars or Restaurants**: Enjoy lunch or an early dinner at a rooftop bar or restaurant to soak in the sun and the city atmosphere.

4. **Botanical Gardens**: The New York Botanical Garden or the Brooklyn Botanic Garden are beautiful places to s